In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from PIL import ImageDraw 
from PIL import ImageFont
import pickle

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
    raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
print(tf.__version__)

1.8.0


In [3]:
# This is needed to display the images.
%matplotlib inline

In [4]:
# What model to download.
MODEL_NAME = 'faster_rcnn_inception_resnet_v2_atrous_coco_11_06_2017'
#'ssd_mobilenet_v1_coco_2017_11_17'
# 


MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

In [5]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())

In [6]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [7]:
from utils import label_map_util

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [10]:
def detect_objects(image_np, sess, detection_graph):
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    # Each box represents a part of the image where a particular object was detected.
    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    # Actual detection.
    (boxes, scores, classes, num_detections) = sess.run(
        [boxes, scores, classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})
    
    classes = np.squeeze(classes)
    boxes = np.squeeze(boxes)
    scores = np.squeeze(scores)
    
    #global boxCenterXprev
    #global boxCenterYprev
    
    # for cars only--------------
    n = len(classes)
    idxs = []
    for i in range(n):
        if classes[i] == 3:
            idxs.append(i)
    
    filtered_boxes = boxes[idxs, ...]
    filtered_scores = scores[idxs, ...]
    filtered_classes = classes[idxs, ...]
    
    #--------------------------------------------------------------
    pkl_file = open(parking_map_file, 'rb')
    mapping_file = pickle.load(pkl_file)
    pkl_file.close()
    global mapping
    mapping = list(mapping_file.values())[0]
    mapping = mapping.tolist()
    mapping = list(map(lambda x: tuple(x[0]), mapping)) 
    mapping.append(mapping[0])
    
    # Visualization of the results of a detection.
    image_np, totNoCar, parked_cars =  vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        mapping,
        filtered_boxes,
        filtered_classes.astype(np.int32),
        filtered_scores,
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8)
    
    #print(totNoCar)
    
    image_pil = Image.fromarray(np.uint8(image_np)).convert('RGB')
    draw = ImageDraw.Draw(image_pil)

    draw.line(mapping, fill="red", width=9)
    for point in mapping:
        draw.ellipse((point[0] - 4, point[1] - 4, point[0]  + 4, point[1] + 4), fill="red")
    
    #--------------------------------------------------------------
    # Count the # of parked cars
    #--------------------------------------------------------------
#     image_np = np.array(image_pil)
#     parked_cars, totCars = vis_util.count_parked_cars(image_np,
#                                     denoised_boxes,
#                                     filtered_classes.astype(np.int32), 
#                                     mapping,
#                                     category_index)
    print(parked_cars)
    #print(totCars)
    print(totNoCar)
    assert(parked_cars <= totNoCar)
    #----------------------------------------------------------------    
    # font = ImageFont.truetype(<font-file>, <font-size>)
    font = ImageFont.truetype('Ubuntu-Bold.ttf', 60)
    dispStr1 = "Total No of parked Cars: "+str(parked_cars)
    #dispStr2 = "Total No of parked Cars: "+str(parked_cars)
    # draw.text((x, y),"sample Text",(r,g,b))
    draw.text((0, 0),dispStr1,(255,255,255),font=font)
    #draw.text((0, 100),dispStr2,(255,255,255),font=font)
    np.copyto(image_np, np.array(image_pil))
    
    return image_np


In [11]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # you should return the final output (image with lines are drawn on lanes)
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            image_process = detect_objects(image, sess, detection_graph)
            return image_process

In [12]:
import visualization_utils as vis_util

In [ ]:
white_output = 'testVideo_out5.mp4'
vidFile = "Data/Parking_demo3.mp4"
parking_map_file = 'Data/mapping_file.pkl'
clip1 = VideoFileClip(vidFile).subclip(10, 15)
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!s
%time white_clip.write_videofile(white_output, audio=False, codec='mpeg4')

0
3
[MoviePy] >>>> Building video testVideo_out5.mp4
[MoviePy] Writing video testVideo_out5.mp4


  1%|          | 1/126 [00:25<52:35, 25.24s/it]

0
3


  2%|▏         | 2/126 [00:50<52:14, 25.28s/it]

0
3


...

0
2


 52%|█████▏    | 66/126 [27:23<24:46, 24.77s/it]

0
2


 53%|█████▎    | 67/126 [27:48<24:19, 24.73s/it]

0
2
